In [1]:
from sqlalchemy import create_engine, Column, Integer, Float, String, Date, Boolean, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.exc import DataError
import dotenv
from dotenv import load_dotenv
import os
import pandas as pd


In [2]:
Base = declarative_base()

C:\Users\arian\AppData\Local\Temp\ipykernel_20764\4196137762.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
devices = pd.read_csv('Devices.csv')
devices.DeviceName.duplicated

<bound method Series.duplicated of 0                   1
1            1 (2021)
2              1 Plus
3                 1.3
4                 1.4
            ...      
2320     nubia Z7 Max
2321    nubia Z7 mini
2322         nubia Z9
2323     nubia Z9 Max
2324    nubia Z9 mini
Name: DeviceName, Length: 2325, dtype: object>

In [4]:
devices[devices['DeviceName'].duplicated()]

,DeviceName,BatterymAh,Price,Chipset,GPU,DisplayType,BrandName,GPRS,EDGE,LaunchDate,...,PPIDensity,OS,CPUCoreCount,MainCamera,SelfieCamera,LoudSpeaker,Jack35,WLAN,NFC,Bluetooth


In [5]:
class Devices(Base):
    __tablename__ = 'devices'
    DeviceID = Column(Integer, primary_key=True)
    BrandName = Column(String)
    DeviceName = Column(String)
    LaunchDate = Column(Integer)
    Status = Column(String)
    Body_x = Column(Float)
    Body_y = Column(Float)
    Body_z = Column(Float)
    Weight = Column(Float)
    BatterymAh = Column(Integer)
    Price = Column(String)
    OS = Column(String)
    Chipset = Column(String)
    CPUCoreCount = Column(Integer)
    GPU = Column(String)
    MainCamera = Column(String)
    SelfieCamera = Column(String)
    WLAN = Column(String)
    Bluetooth = Column(String)
    NFC = Column(Boolean)
    LoadSpeaker = Column(Boolean)
    Jack35 = Column(Boolean)
    DisplayType = Column(String)
    DisplaySize = Column(Float)
    DisplayArea = Column(Float)
    ScreenToBodyRatio = Column(String)
    PixelX = Column(Integer)
    PixelY = Column(Integer)
    PPIDensity = Column(String)
    AdditionalInfo = Column(String)
    # Relationships
    sim = relationship("SIM", back_populates="device")
    memory_specifications = relationship("MemorySpecifications", back_populates="device")
    device_technology = relationship("DeviceTechnology", back_populates="device")
    device_networks = relationship("DeviceNetworks", back_populates="device")
    usb_specifications = relationship("USBSpecifications", back_populates="device")
    feature_specifications = relationship("FeatureSpecifications", back_populates="device")

class SIM(Base):
    __tablename__ = 'sim'
    ID = Column(Integer, primary_key=True, autoincrement=True)
    DeviceID = Column(Integer, ForeignKey('devices.DeviceID'))
    Name = Column(String)
    device = relationship("Devices", back_populates="sim")

class MemorySpecifications(Base):
    __tablename__ = 'memory_specifications'
    ID = Column(Integer, primary_key=True, autoincrement=True)
    DeviceID = Column(Integer, ForeignKey('devices.DeviceID'))
    Storage = Column(String)
    RAM = Column(String)
    device = relationship("Devices", back_populates="memory_specifications")

class DeviceTechnology(Base):
    __tablename__ = 'device_technology'
    ID = Column(Integer, primary_key=True, autoincrement=True)
    DeviceID = Column(Integer, ForeignKey('devices.DeviceID'))
    Technology = Column(String)
    device = relationship("Devices", back_populates="device_technology")

class DeviceNetworks(Base):
    __tablename__ = 'device_networks'
    ID = Column(Integer, primary_key=True, autoincrement=True)
    DeviceID = Column(Integer, ForeignKey('devices.DeviceID'))
    Network = Column(String)
    device = relationship("Devices", back_populates="device_networks")

class USBSpecifications(Base):
    __tablename__ = 'usb_specifications'
    ID = Column(Integer, primary_key=True, autoincrement=True)
    DeviceID = Column(Integer, ForeignKey('devices.DeviceID'))
    USB = Column(String)
    device = relationship("Devices", back_populates="usb_specifications")

class FeatureSpecifications(Base):
    __tablename__ = 'feature_specifications'
    ID = Column(Integer, primary_key=True, autoincrement=True)
    DeviceID = Column(Integer, ForeignKey('devices.DeviceID'))
    Sensor = Column(String)
    device = relationship("Devices", back_populates="feature_specifications")


In [6]:
load_dotenv()

url = os.getenv('DATABASE_URL')
engine = create_engine(url)
Base.metadata.create_all(engine)

In [9]:
Session = sessionmaker(bind=engine)
session = Session()

def process_numeric_column(value):
    """
    Processes a numeric column that may contain values with commas as thousand separators
    and/or values like '4000/5050', selecting the larger of two numbers if slash-separated.
    Returns an integer after cleaning or the larger of two if applicable.
    """
    # Remove commas for thousands
    value = str(value).replace(',', '')
    
    # Split on '/' and take the max if applicable
    if '/' in value:
        parts = value.split('/')
        numbers = [int(part) for part in parts if part.isdigit()]
        return max(numbers) if numbers else None
    return int(value) if value.isdigit() else None

    
def insert_devices_and_create_id_mapping(csv_path, session, model):
    df = pd.read_csv(csv_path)

    if 'BatterymAh' in df.columns:
        df['BatterymAh'] = df['BatterymAh'].apply(lambda x: process_numeric_column(x))

    model_columns = {column.name for column in model.__table__.columns}
    
    # Correctly handle data types here if necessary
    # For example, if some columns should be integers or floats but are represented as strings in the CSV
    # df['SomeColumn'] = pd.to_numeric(df['SomeColumn'], errors='coerce')

    filtered_df = df[[col for col in df.columns if col in model_columns]]
    name_id_mapping = {}
    
    for _, row in filtered_df.iterrows():
        # Handle conversion of specific columns to the correct data types if needed
        try:
            device_instance = model(**row.to_dict())
            session.add(device_instance)
            session.commit()  # Committing inside the loop to ensure IDs are generated
            # Retrieve and map the DeviceName to DeviceID immediately after insertion to handle non-unique names
            if 'DeviceName' in row:
                name_id_mapping[row['DeviceName']] = device_instance.DeviceID
        except DataError as e:
            print(f"Error inserting device: {e}")
            session.rollback()

    return name_id_mapping

def load_csv_with_device_id(csv_path, model, session, name_id_mapping, device_name_column='DeviceName'):
    df = pd.read_csv(csv_path)
    if device_name_column in df.columns:
        df['DeviceID'] = df[device_name_column].map(name_id_mapping)
        df.drop(columns=[device_name_column], inplace=True)
        
    # Convert all other columns to string, excluding DeviceID
    columns_to_convert = df.columns.difference(['DeviceID'])
    df[columns_to_convert] = df[columns_to_convert].astype(str)
        
    
    for _, row in df.iterrows():
        try:
            instance = model(**row.to_dict())
            session.add(instance)
        except DataError as e:
            print(f"Error loading data: {e}")
            session.rollback()
    session.commit()

# Example usage
devices_csv_path = "Devices.csv"
device_name_id_mapping = insert_devices_and_create_id_mapping(devices_csv_path, session, Devices)

# Assuming SIM, MemorySpecifications, etc., have been correctly defined in your SQLAlchemy models
csv_paths_and_models = [
    ("SIM.csv", SIM, 'DeviceName'),
    ("Network.csv", DeviceNetworks, 'DeviceName'),
    ("Technology.csv", DeviceTechnology , 'DeviceName'),
    ("USB.csv", USBSpecifications, 'DeviceName'),
    ("Sensor.csv", FeatureSpecifications, 'DeviceName'),
    ("Memory.csv", MemorySpecifications, 'DeviceName'),

]

for csv_path, model, device_name_column in csv_paths_and_models:
    load_csv_with_device_id(csv_path, model, session, device_name_id_mapping, device_name_column)

session.close()

DataError: (psycopg2.errors.InvalidTextRepresentation) invalid input syntax for type double precision: "OTG"
LINE 1: ...LECT p0::INTEGER, p1::VARCHAR FROM (VALUES (3331, 'OTG', 0),...
                                                             ^

[SQL: INSERT INTO usb_specifications ("DeviceID", "USB") SELECT p0::INTEGER, p1::VARCHAR FROM (VALUES (%(DeviceID__0)s, %(USB__0)s, 0), (%(DeviceID__1)s, %(USB__1)s, 1), (%(DeviceID__2)s, %(USB__2)s, 2), (%(DeviceID__3)s, %(USB__3)s, 3), (%(DeviceID__4)s,  ... 39538 characters truncated ... _counter) ORDER BY sen_counter RETURNING usb_specifications."ID", usb_specifications."ID" AS "ID__1"]
[parameters: {'USB__0': 'OTG', 'DeviceID__0': 3331, 'USB__1': 'USB Type-C 2.0', 'DeviceID__1': 4510, 'USB__2': 'OTG', 'DeviceID__2': 4510, 'USB__3': 'USB Type-C 2.0', 'DeviceID__3': 3659, 'USB__4': 'OTG', 'DeviceID__4': 3659, 'USB__5': 'Yes', 'DeviceID__5': 3328, 'USB__6': 'microUSB 2.0', 'DeviceID__6': 4092, 'USB__7': 'OTG', 'DeviceID__7': 4092, 'USB__8':